In [ ]:
using PerlaTonettiWaugh, LinearAlgebra, Plots, BenchmarkTools, Interpolations

Set up parameters and find the corresponding stationary solution:

In [ ]:
z_min = 0.0 
z_max = 5.0
M = 1000
z_grid = range(z_min, stop = z_max, length = M) # Since we only care about the grid. 

# Define common objects. 
d_0 = 5
d_T = 2.3701
params = (ρ = 0.02, σ = 4.2508, N = 10, θ = 5.1269, γ = 1.00, κ = 0.013, ζ = 1, η = 0, Theta = 1, χ = 1/(2.1868), υ = 0.0593, μ = 0, δ = 0.053) # Baselines per Jesse. 

# solve for stationary solution at t = 0
params_0 = merge(params, (d = d_0,)) # parameters to be used at t = 0
params_T = merge(params, (d = d_T,)) # parameters to be used at t = T

stationary_sol_0 = stationary_numerical(params_0, z_grid) # solution at t = 0
stationary_sol = stationary_numerical(params_T, z_grid) # solution at t = T

Ω_0 = stationary_sol_0.Ω
Ω_T = stationary_sol.Ω

println("g = $(stationary_sol.g), z_hat = $(stationary_sol.z_hat), Ω = $(stationary_sol.Ω)")

Choose `T` and `Ω_vec0` with its corresponding nodes `Ω_nodes`:

In [ ]:
# control parameter(s) for Ω and E
T = 25.0
c_1 = 0.0
c_2 = 0.0

In [ ]:
# quadratic E
δ = params.δ
a = (c_1*T^3 + 3*T*δ - 6*log(Ω_0) + 6*log(Ω_T)) / (3*T)
b = 2*(2*c_1*T^3 - 3*log(Ω_0) + 3*log(Ω_T)) / (3*T*T)

Ω = t -> exp( t*((t-T)^2)*(4*c_1+3*c_2*(t+2*T)) /12 ) * Ω_0 * (Ω_0/Ω_T)^(t*(t-2.0*T)/(T*T))
E = t -> (12*t*log(Ω_0/Ω_T) + T*( (t-T)*T*(6*t*(c_1+c_2*t) - 2*(c_1-3*c_2*t)*T - 3*c_2*T*T)  + 12*log(Ω_T/Ω_0) ) ) / (6*T*T) + δ

In [ ]:
# check if Ω and E make sense
@assert Ω(0) ≈ Ω_0
@assert Ω(T) ≈ Ω_T
@assert E(T) ≈ δ

settings = (z = z_grid, tstops = 0:1e-3:T, Δ_E = 1e-06)
entry_residuals_nodes = 1e-1:1e-1:(T-1e-1)

Return the entry residuals:

In [ ]:
# solve the dynamics!
@time solved_tuple = solve_dynamics(params_T, stationary_sol, settings, T, Ω, E)
solved = solved_tuple.results
v_t0 = solved_tuple.sol.u[1][1:M]
v0 = solved[:v_0]

# save v0 and v_hat_t0
v_hat_t0 = map(z -> exp((params.σ-1)*z), z_grid) .* v_t0;

## Plots for Ω and `entry_residuals`

In [ ]:
plot_Ω = plot(solved.t, Ω.(solved.t), label = "Omega", lw = 3)
plot_residual = plot(solved.t, solved.entry_residual, label = "entry_residual", lw = 3)
plot(plot_Ω, plot_residual, layout = (2,1))

In [ ]:
norm(solved.entry_residual)

## Primary Plots

In [ ]:
plot1 = plot(solved.t, solved.g, label = "g", lw = 3)
plot2 = plot(solved.t, solved.z_hat, label = "z_hat", lw = 3)
plot3 = plot(solved.t, solved.S, label = "S", lw = 3)
plot4 = plot(solved.t, solved.entry_residual, label = "entry_residual", lw = 3)
plot(plot1, plot2, plot3, plot4, layout=(2,2))

## Static Equations

In [ ]:
plot1 = plot(solved.t, solved.L_tilde, label = "L_tilde", lw = 3)
plot2 = plot(solved.t, solved.z_bar, label = "z_bar", lw = 3)
plot3 = plot(solved.t, solved.π_min, label = "pi_min", lw = 3)
plot4 = plot(solved.t, solved.λ_ii, label = "lambda_ii", lw = 3)
plot5 = plot(solved.t, solved.c, label = "c", lw = 3)
plot6 = plot(solved.t, solved.E, label = "E", lw = 3)
plot7 = plot(z_grid, v_hat_t0, label = "v_hat at t = 0", lw = 3)
plot(plot1, plot2, plot3, plot4, plot5, plot6, layout=(3,2))

In [ ]:
# Can examine the returned data with the Voyager and/or Vegalite
using DataVoyager, VegaLite
solved |> Voyager()
#solved |> @vlplot(:line, x = :t, y = :g, width=400, height=400)